# Flight data in Brazil

## Import libraries and data

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier

In [2]:
# Check for the working directory
!pwd

/Users/escortkwon/Code/Portfolio_Project_2021/Flights in Brazil


In [3]:
# Check for what files does certain directory have
!ls

FlightsAnalysis.ipynb archive.zip
airport-codes.csv     raw_2021.csv


In [4]:
# Unzip for fetch 'raw_2021.csv'
!unzip -o archive.zip

Archive:  archive.zip
  inflating: raw_2021.csv            


In [5]:
# Load csv file into DataFrame
flights = pd.read_csv('raw_2021.csv', encoding='utf-8', low_memory=False)
flights

,company,flight,auth_code,line_type,origin,destination,scheduled_dep,real_dep,scheduled_arr,real_arr,situation,jus_code
0,AZU,2497,0,N,SBMO,SBRF,NaN,NaN,2019-06-07 21:50:00,2019-06-07 21:39:00,REALIZADO,NaN
1,AZU,2493,0,N,SBMK,SBCF,NaN,NaN,2019-06-07 20:30:00,2019-06-07 20:30:00,REALIZADO,NaN
2,AZU,2489,0,N,SBRF,SBGR,NaN,NaN,2019-06-07 23:35:00,2019-06-07 23:35:00,REALIZADO,NaN
3,AZU,2488,0,N,SBGR,SBRF,NaN,NaN,2019-06-07 08:20:00,2019-06-07 08:11:00,REALIZADO,NaN
4,AZU,2487,0,N,SBFL,SBKP,NaN,NaN,2019-06-07 17:45:00,2019-06-07 17:45:00,REALIZADO,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1592596,AZU,2503,0,N,SBPL,SBKP,NaN,NaN,2019-06-07 06:35:00,2019-06-07 06:31:00,REALIZADO,NaN
1592597,AZU,2502,0,N,SBRF,SBSG,NaN,NaN,2019-06-07 15:25:00,2019-06-07 15:05:00,REALIZADO,HD
1592598,AZU,2500,0,N,SBKP,SBPL,NaN,NaN,2019-06-08 02:35:00,2019-06-08 02:18:00,REALIZADO,HD
1592599,AZU,2499,0,N,SBJU,SBKP,NaN,NaN,2019-06-07 05:25:00,2019-06-07 05:09:00,REALIZADO,HD


In [32]:
# Load csv file into DataFrame
airports = pd.read_csv('airport-codes.csv', engine='python', encoding='utf-8')
airports

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
0,00A,heliport,Total Rf Heliport,11.0,NaN,US,US-PA,Bensalem,00A,NaN,00A,"-74.93360137939453, 40.07080078125"
1,00AA,small_airport,Aero B Ranch Airport,3435.0,NaN,US,US-KS,Leoti,00AA,NaN,00AA,"-101.473911, 38.704022"
2,00AK,small_airport,Lowell Field,450.0,NaN,US,US-AK,Anchor Point,00AK,NaN,00AK,"-151.695999146, 59.94919968"
3,00AL,small_airport,Epps Airpark,820.0,NaN,US,US-AL,Harvest,00AL,NaN,00AL,"-86.77030181884766, 34.86479949951172"
4,00AR,closed,Newport Hospital & Clinic Heliport,237.0,NaN,US,US-AR,Newport,NaN,NaN,NaN,"-91.254898, 35.6087"
...,...,...,...,...,...,...,...,...,...,...,...,...
57416,ZYYK,medium_airport,Yingkou Lanqi Airport,0.0,AS,CN,CN-21,Yingkou,ZYYK,YKH,NaN,"122.3586, 40.542524"
57417,ZYYY,medium_airport,Shenyang Dongta Airport,NaN,AS,CN,CN-21,Shenyang,ZYYY,NaN,NaN,"123.49600219726562, 41.784400939941406"
57418,ZZ-0001,heliport,Sealand Helipad,40.0,EU,GB,GB-ENG,Sealand,NaN,NaN,NaN,"1.4825, 51.894444"
57419,ZZ-0002,small_airport,Glorioso Islands Airstrip,11.0,AF,TF,TF-U-A,Grande Glorieuse,NaN,NaN,NaN,"47.296388888900005, -11.584277777799999"


## Data Preprocessing

### Flights

In [7]:
# Check null values by columns
flights.isnull().sum()

company                0
flight                 0
auth_code            636
line_type           1525
origin                 0
destination            0
scheduled_dep    1344296
real_dep         1356256
scheduled_arr       2474
real_arr           46380
situation              0
jus_code         1185571
dtype: int64

In [8]:
# Drop unnecesary columns
flights.drop(['auth_code', 'scheduled_dep', 'real_dep', 'situation', 'jus_code'], axis=1, inplace=True)
flights

,company,flight,line_type,origin,destination,scheduled_arr,real_arr
0,AZU,2497,N,SBMO,SBRF,2019-06-07 21:50:00,2019-06-07 21:39:00
1,AZU,2493,N,SBMK,SBCF,2019-06-07 20:30:00,2019-06-07 20:30:00
2,AZU,2489,N,SBRF,SBGR,2019-06-07 23:35:00,2019-06-07 23:35:00
3,AZU,2488,N,SBGR,SBRF,2019-06-07 08:20:00,2019-06-07 08:11:00
4,AZU,2487,N,SBFL,SBKP,2019-06-07 17:45:00,2019-06-07 17:45:00
...,...,...,...,...,...,...,...
1592596,AZU,2503,N,SBPL,SBKP,2019-06-07 06:35:00,2019-06-07 06:31:00
1592597,AZU,2502,N,SBRF,SBSG,2019-06-07 15:25:00,2019-06-07 15:05:00
1592598,AZU,2500,N,SBKP,SBPL,2019-06-08 02:35:00,2019-06-08 02:18:00
1592599,AZU,2499,N,SBJU,SBKP,2019-06-07 05:25:00,2019-06-07 05:09:00


In [9]:
# Remove rows which has nulll values
flights.dropna(axis=0, inplace=True)
flights

,company,flight,line_type,origin,destination,scheduled_arr,real_arr
0,AZU,2497,N,SBMO,SBRF,2019-06-07 21:50:00,2019-06-07 21:39:00
1,AZU,2493,N,SBMK,SBCF,2019-06-07 20:30:00,2019-06-07 20:30:00
2,AZU,2489,N,SBRF,SBGR,2019-06-07 23:35:00,2019-06-07 23:35:00
3,AZU,2488,N,SBGR,SBRF,2019-06-07 08:20:00,2019-06-07 08:11:00
4,AZU,2487,N,SBFL,SBKP,2019-06-07 17:45:00,2019-06-07 17:45:00
...,...,...,...,...,...,...,...
1592596,AZU,2503,N,SBPL,SBKP,2019-06-07 06:35:00,2019-06-07 06:31:00
1592597,AZU,2502,N,SBRF,SBSG,2019-06-07 15:25:00,2019-06-07 15:05:00
1592598,AZU,2500,N,SBKP,SBPL,2019-06-08 02:35:00,2019-06-08 02:18:00
1592599,AZU,2499,N,SBJU,SBKP,2019-06-07 05:25:00,2019-06-07 05:09:00


In [10]:
# Re-check DataFRame whether it contains null rows or not
flights.isnull().sum()

company          0
flight           0
line_type        0
origin           0
destination      0
scheduled_arr    0
real_arr         0
dtype: int64

### Airports

In [33]:
airports

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
0,00A,heliport,Total Rf Heliport,11.0,NaN,US,US-PA,Bensalem,00A,NaN,00A,"-74.93360137939453, 40.07080078125"
1,00AA,small_airport,Aero B Ranch Airport,3435.0,NaN,US,US-KS,Leoti,00AA,NaN,00AA,"-101.473911, 38.704022"
2,00AK,small_airport,Lowell Field,450.0,NaN,US,US-AK,Anchor Point,00AK,NaN,00AK,"-151.695999146, 59.94919968"
3,00AL,small_airport,Epps Airpark,820.0,NaN,US,US-AL,Harvest,00AL,NaN,00AL,"-86.77030181884766, 34.86479949951172"
4,00AR,closed,Newport Hospital & Clinic Heliport,237.0,NaN,US,US-AR,Newport,NaN,NaN,NaN,"-91.254898, 35.6087"
...,...,...,...,...,...,...,...,...,...,...,...,...
57416,ZYYK,medium_airport,Yingkou Lanqi Airport,0.0,AS,CN,CN-21,Yingkou,ZYYK,YKH,NaN,"122.3586, 40.542524"
57417,ZYYY,medium_airport,Shenyang Dongta Airport,NaN,AS,CN,CN-21,Shenyang,ZYYY,NaN,NaN,"123.49600219726562, 41.784400939941406"
57418,ZZ-0001,heliport,Sealand Helipad,40.0,EU,GB,GB-ENG,Sealand,NaN,NaN,NaN,"1.4825, 51.894444"
57419,ZZ-0002,small_airport,Glorioso Islands Airstrip,11.0,AF,TF,TF-U-A,Grande Glorieuse,NaN,NaN,NaN,"47.296388888900005, -11.584277777799999"


In [34]:
# Split coordinates column into latitude and longtitude
airports['long'] = airports['coordinates'].str.split(',', expand=True)[0]
airports['lat'] = airports['coordinates'].str.split(',', expand=True)[1]
airports

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates,long,lat
0,00A,heliport,Total Rf Heliport,11.0,NaN,US,US-PA,Bensalem,00A,NaN,00A,"-74.93360137939453, 40.07080078125",-74.93360137939453,40.07080078125
1,00AA,small_airport,Aero B Ranch Airport,3435.0,NaN,US,US-KS,Leoti,00AA,NaN,00AA,"-101.473911, 38.704022",-101.473911,38.704022
2,00AK,small_airport,Lowell Field,450.0,NaN,US,US-AK,Anchor Point,00AK,NaN,00AK,"-151.695999146, 59.94919968",-151.695999146,59.94919968
3,00AL,small_airport,Epps Airpark,820.0,NaN,US,US-AL,Harvest,00AL,NaN,00AL,"-86.77030181884766, 34.86479949951172",-86.77030181884766,34.86479949951172
4,00AR,closed,Newport Hospital & Clinic Heliport,237.0,NaN,US,US-AR,Newport,NaN,NaN,NaN,"-91.254898, 35.6087",-91.254898,35.6087
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57416,ZYYK,medium_airport,Yingkou Lanqi Airport,0.0,AS,CN,CN-21,Yingkou,ZYYK,YKH,NaN,"122.3586, 40.542524",122.3586,40.542524
57417,ZYYY,medium_airport,Shenyang Dongta Airport,NaN,AS,CN,CN-21,Shenyang,ZYYY,NaN,NaN,"123.49600219726562, 41.784400939941406",123.49600219726562,41.784400939941406
57418,ZZ-0001,heliport,Sealand Helipad,40.0,EU,GB,GB-ENG,Sealand,NaN,NaN,NaN,"1.4825, 51.894444",1.4825,51.894444
57419,ZZ-0002,small_airport,Glorioso Islands Airstrip,11.0,AF,TF,TF-U-A,Grande Glorieuse,NaN,NaN,NaN,"47.296388888900005, -11.584277777799999",47.296388888900005,-11.584277777799999


In [35]:
# Drop unnecessary columns
airports.drop(['type', 'elevation_ft', 'continent', 'municipality', 'gps_code', 'iata_code', 'local_code', 'coordinates'], axis=1, inplace=True)
airports

,ident,name,iso_country,iso_region,long,lat
0,00A,Total Rf Heliport,US,US-PA,-74.93360137939453,40.07080078125
1,00AA,Aero B Ranch Airport,US,US-KS,-101.473911,38.704022
2,00AK,Lowell Field,US,US-AK,-151.695999146,59.94919968
3,00AL,Epps Airpark,US,US-AL,-86.77030181884766,34.86479949951172
4,00AR,Newport Hospital & Clinic Heliport,US,US-AR,-91.254898,35.6087
...,...,...,...,...,...,...
57416,ZYYK,Yingkou Lanqi Airport,CN,CN-21,122.3586,40.542524
57417,ZYYY,Shenyang Dongta Airport,CN,CN-21,123.49600219726562,41.784400939941406
57418,ZZ-0001,Sealand Helipad,GB,GB-ENG,1.4825,51.894444
57419,ZZ-0002,Glorioso Islands Airstrip,TF,TF-U-A,47.296388888900005,-11.584277777799999


In [36]:
# Relocate columns and rename 'ident'
airports.rename(columns={'ident': 'ICAO'}, inplace=True)
airports = airports[['ICAO', 'name', 'iso_country', 'iso_region', 'lat', 'long']]
airports

,ICAO,name,iso_country,iso_region,lat,long
0,00A,Total Rf Heliport,US,US-PA,40.07080078125,-74.93360137939453
1,00AA,Aero B Ranch Airport,US,US-KS,38.704022,-101.473911
2,00AK,Lowell Field,US,US-AK,59.94919968,-151.695999146
3,00AL,Epps Airpark,US,US-AL,34.86479949951172,-86.77030181884766
4,00AR,Newport Hospital & Clinic Heliport,US,US-AR,35.6087,-91.254898
...,...,...,...,...,...,...
57416,ZYYK,Yingkou Lanqi Airport,CN,CN-21,40.542524,122.3586
57417,ZYYY,Shenyang Dongta Airport,CN,CN-21,41.784400939941406,123.49600219726562
57418,ZZ-0001,Sealand Helipad,GB,GB-ENG,51.894444,1.4825
57419,ZZ-0002,Glorioso Islands Airstrip,TF,TF-U-A,-11.584277777799999,47.296388888900005


## Data Preparing

In [13]:
# Check unique values of each column
for i in range(len(flights.columns)):
    column_list = flights.columns.tolist()
    print(column_list[i])
    print(flights.iloc[:, i].unique())

company
['AZU' 'UAE' 'TTL' 'TPU' 'THY' 'TCV' 'TAP' 'TAM' 'QTR' 'PTB' 'PAM' 'NRS'
 'MWM' 'MPH' 'LTG' 'LPE' 'LCO' 'LAP' 'LAN' 'UPS' 'UAL' 'LAE' 'KLM' 'KAL'
 'IBE' 'GTI' 'GLO' 'SWR' 'SKU' 'SAA' 'RAM' 'GEC' 'FDX' 'ETH' 'EDW' 'DTA'
 'DSM' 'DLH' 'DAL' 'CMP' 'CLX' 'CCA' 'BOV' 'BAW' 'AZA' 'AVA' 'AUT' 'ARG'
 'AMX' 'AFR' 'AEA' 'ACA' 'AAL' 'AAF' 'TPA' 'CFG' 'AZP' 'OWT' 'SLM' 'ROI'
 'FBZ' 'AZN' 'SUL' 'JAT' 'SID' 'AUZ' 'ELY' 'EVE' 'MMZ' 'CKS' 'TVS' 'SWQ'
 'MSI' 'MAA' 'AWC' 'KMF' 'VIV' 'CSB' 'OLS' 'VVC' 'EDR' 'TIW' 'WAL' 'ACL'
 'ARL' 'ARE' 'TVR' 'SKX' 'PVV' 'ETD' 'RZO' 'VDA' 'SOO' 'ADB' 'ETR' 'AXY'
 'LYC' 'SAS' 'LAU' 'TGY' 'EAL' 'PVG' 'JAF' 'TFL' 'ATG' 'KRE' 'RUC' 'QCL'
 'AJB' 'PUE' 'IPM' 'LNE' 'ONE']
flight
['2497' '2493' '2489' ... '9848' '7092' '9850']
line_type
['N' 'I' 'C' 'G' 'L' 'R' 'E' 'H' 'X']
origin
['SBMO' 'SBMK' 'SBRF' 'SBGR' 'SBFL' 'SBPK' 'SBCF' 'SBKP' 'SBLO' 'SBCY'
 'SBCT' 'SBRJ' 'SBSP' 'SNBR' 'SAEZ' 'SBEG' 'SBUY' 'SWCA' 'SPJC' 'SBGL'
 'SBPA' 'LTFM' 'GVAC' 'SBSV' 'LPPT' 'LPPR' 'SBFZ' '

In [14]:
# Copy flights for backup
flights_copy = flights.copy()
flights_copy

,company,flight,line_type,origin,destination,scheduled_arr,real_arr
0,AZU,2497,N,SBMO,SBRF,2019-06-07 21:50:00,2019-06-07 21:39:00
1,AZU,2493,N,SBMK,SBCF,2019-06-07 20:30:00,2019-06-07 20:30:00
2,AZU,2489,N,SBRF,SBGR,2019-06-07 23:35:00,2019-06-07 23:35:00
3,AZU,2488,N,SBGR,SBRF,2019-06-07 08:20:00,2019-06-07 08:11:00
4,AZU,2487,N,SBFL,SBKP,2019-06-07 17:45:00,2019-06-07 17:45:00
...,...,...,...,...,...,...,...
1592596,AZU,2503,N,SBPL,SBKP,2019-06-07 06:35:00,2019-06-07 06:31:00
1592597,AZU,2502,N,SBRF,SBSG,2019-06-07 15:25:00,2019-06-07 15:05:00
1592598,AZU,2500,N,SBKP,SBPL,2019-06-08 02:35:00,2019-06-08 02:18:00
1592599,AZU,2499,N,SBJU,SBKP,2019-06-07 05:25:00,2019-06-07 05:09:00


In [15]:
# I have no idea with line_type of following values: C, G, L, R, E, H and X
# Therefore, I decided to remove rows which contain those
target = ['N', 'I']
flights = flights_copy.loc[flights['line_type'].isin(target)]
flights

,company,flight,line_type,origin,destination,scheduled_arr,real_arr
0,AZU,2497,N,SBMO,SBRF,2019-06-07 21:50:00,2019-06-07 21:39:00
1,AZU,2493,N,SBMK,SBCF,2019-06-07 20:30:00,2019-06-07 20:30:00
2,AZU,2489,N,SBRF,SBGR,2019-06-07 23:35:00,2019-06-07 23:35:00
3,AZU,2488,N,SBGR,SBRF,2019-06-07 08:20:00,2019-06-07 08:11:00
4,AZU,2487,N,SBFL,SBKP,2019-06-07 17:45:00,2019-06-07 17:45:00
...,...,...,...,...,...,...,...
1592596,AZU,2503,N,SBPL,SBKP,2019-06-07 06:35:00,2019-06-07 06:31:00
1592597,AZU,2502,N,SBRF,SBSG,2019-06-07 15:25:00,2019-06-07 15:05:00
1592598,AZU,2500,N,SBKP,SBPL,2019-06-08 02:35:00,2019-06-08 02:18:00
1592599,AZU,2499,N,SBJU,SBKP,2019-06-07 05:25:00,2019-06-07 05:09:00


In [16]:
# Check unique values of each column
for i in range(len(flights.columns)):
    column_list = flights.columns.tolist()
    print(column_list[i])
    print(flights.iloc[:, i].unique())

company
['AZU' 'UAE' 'TTL' 'TPU' 'THY' 'TCV' 'TAP' 'TAM' 'QTR' 'PTB' 'PAM' 'NRS'
 'LPE' 'LAP' 'LAN' 'UAL' 'KLM' 'KAL' 'IBE' 'GLO' 'SWR' 'SKU' 'SAA' 'RAM'
 'ETH' 'EDW' 'DTA' 'DSM' 'DLH' 'DAL' 'CMP' 'CCA' 'BOV' 'BAW' 'AZA' 'AVA'
 'AUT' 'ARG' 'AMX' 'AFR' 'AEA' 'ACA' 'AAL' 'AAF' 'CFG' 'AZP' 'OWT' 'SLM'
 'ROI' 'LTG' 'FBZ' 'CLX' 'AZN' 'SUL' 'JAT' 'AUZ' 'SID' 'ELY' 'IPM' 'ONE']
flight
['2497' '2493' '2489' ... '9848' '7092' '9850']
line_type
['N' 'I']
origin
['SBMO' 'SBMK' 'SBRF' 'SBGR' 'SBFL' 'SBPK' 'SBCF' 'SBKP' 'SBLO' 'SBCY'
 'SBCT' 'SBRJ' 'SBSP' 'SNBR' 'SAEZ' 'SBEG' 'SBUY' 'SWCA' 'SPJC' 'SBGL'
 'SBPA' 'LTFM' 'GVAC' 'SBSV' 'LPPT' 'LPPR' 'SBFZ' 'SBSN' 'SBPS' 'SBTC'
 'SBCG' 'SBBR' 'SBSL' 'SBBE' 'KMIA' 'KMCO' 'KJFK' 'KBOS' 'SCEL' 'LEBL'
 'MMMX' 'LIRF' 'LFPG' 'EGLL' 'EDDF' 'LEMD' 'LIMC' 'FAOR' 'SUMU' 'SKBO'
 'SBUL' 'SBGO' 'SBMA' 'SBFI' 'SBBV' 'SBSG' 'SBJP' 'SBAR' 'SBPJ' 'OTHH'
 'SBRP' 'SWGN' 'SBQV' 'SWKO' 'SBTF' 'SWEI' 'SWPI' 'SBIH' 'KLAX' 'SGAS'
 'KIAD' 'KORD' 'KEWR' 'KIAH' 'OMDB' 'EHAM' 'PAN

In [17]:
# Split Arrival(Scheduled, Real) into Year, Month and Date
flights['scheduled_arr'] = pd.to_datetime(flights['scheduled_arr'])
flights['real_arr'] = pd.to_datetime(flights['real_arr'])
flights.dtypes

/var/folders/rl/qrb5zxv57qg8121z8dhpdklh0000gn/T/ipykernel_7877/2228701292.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  flights['scheduled_arr'] = pd.to_datetime(flights['scheduled_arr'])
/var/folders/rl/qrb5zxv57qg8121z8dhpdklh0000gn/T/ipykernel_7877/2228701292.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  flights['real_arr'] = pd.to_datetime(flights['real_arr'])


company                  object
flight                   object
line_type                object
origin                   object
destination              object
scheduled_arr    datetime64[ns]
real_arr         datetime64[ns]
dtype: object

In [18]:
# Extract values of year, month and day by using attribute of dt
flights['scheduled_arr_Year'] = flights['scheduled_arr'].dt.year
flights['scheduled_arr_Month'] = flights['scheduled_arr'].dt.month
flights['scheduled_arr_Day'] = flights['scheduled_arr'].dt.day
flights['real_arr_Year'] = flights['real_arr'].dt.year
flights['real_arr_Month'] = flights['real_arr'].dt.month
flights['real_arr_Day'] = flights['real_arr'].dt.day

flights

/var/folders/rl/qrb5zxv57qg8121z8dhpdklh0000gn/T/ipykernel_7877/992535643.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  flights['scheduled_arr_Year'] = flights['scheduled_arr'].dt.year
/var/folders/rl/qrb5zxv57qg8121z8dhpdklh0000gn/T/ipykernel_7877/992535643.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  flights['scheduled_arr_Month'] = flights['scheduled_arr'].dt.month
/var/folders/rl/qrb5zxv57qg8121z8dhpdklh0000gn/T/ipykernel_7877/992535643.py:4: SettingWithCopyWarning: 
A value is trying to be

,company,flight,line_type,origin,destination,scheduled_arr,real_arr,scheduled_arr_Year,scheduled_arr_Month,scheduled_arr_Day,real_arr_Year,real_arr_Month,real_arr_Day
0,AZU,2497,N,SBMO,SBRF,2019-06-07 21:50:00,2019-06-07 21:39:00,2019,6,7,2019,6,7
1,AZU,2493,N,SBMK,SBCF,2019-06-07 20:30:00,2019-06-07 20:30:00,2019,6,7,2019,6,7
2,AZU,2489,N,SBRF,SBGR,2019-06-07 23:35:00,2019-06-07 23:35:00,2019,6,7,2019,6,7
3,AZU,2488,N,SBGR,SBRF,2019-06-07 08:20:00,2019-06-07 08:11:00,2019,6,7,2019,6,7
4,AZU,2487,N,SBFL,SBKP,2019-06-07 17:45:00,2019-06-07 17:45:00,2019,6,7,2019,6,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1592596,AZU,2503,N,SBPL,SBKP,2019-06-07 06:35:00,2019-06-07 06:31:00,2019,6,7,2019,6,7
1592597,AZU,2502,N,SBRF,SBSG,2019-06-07 15:25:00,2019-06-07 15:05:00,2019,6,7,2019,6,7
1592598,AZU,2500,N,SBKP,SBPL,2019-06-08 02:35:00,2019-06-08 02:18:00,2019,6,8,2019,6,8
1592599,AZU,2499,N,SBJU,SBKP,2019-06-07 05:25:00,2019-06-07 05:09:00,2019,6,7,2019,6,7


In [19]:
# Check unique values of each column
for i in range(len(flights.columns)):
    column_list = flights.columns.tolist()
    print(column_list[i])
    print(flights.iloc[:, i].unique())

company
['AZU' 'UAE' 'TTL' 'TPU' 'THY' 'TCV' 'TAP' 'TAM' 'QTR' 'PTB' 'PAM' 'NRS'
 'LPE' 'LAP' 'LAN' 'UAL' 'KLM' 'KAL' 'IBE' 'GLO' 'SWR' 'SKU' 'SAA' 'RAM'
 'ETH' 'EDW' 'DTA' 'DSM' 'DLH' 'DAL' 'CMP' 'CCA' 'BOV' 'BAW' 'AZA' 'AVA'
 'AUT' 'ARG' 'AMX' 'AFR' 'AEA' 'ACA' 'AAL' 'AAF' 'CFG' 'AZP' 'OWT' 'SLM'
 'ROI' 'LTG' 'FBZ' 'CLX' 'AZN' 'SUL' 'JAT' 'AUZ' 'SID' 'ELY' 'IPM' 'ONE']
flight
['2497' '2493' '2489' ... '9848' '7092' '9850']
line_type
['N' 'I']
origin
['SBMO' 'SBMK' 'SBRF' 'SBGR' 'SBFL' 'SBPK' 'SBCF' 'SBKP' 'SBLO' 'SBCY'
 'SBCT' 'SBRJ' 'SBSP' 'SNBR' 'SAEZ' 'SBEG' 'SBUY' 'SWCA' 'SPJC' 'SBGL'
 'SBPA' 'LTFM' 'GVAC' 'SBSV' 'LPPT' 'LPPR' 'SBFZ' 'SBSN' 'SBPS' 'SBTC'
 'SBCG' 'SBBR' 'SBSL' 'SBBE' 'KMIA' 'KMCO' 'KJFK' 'KBOS' 'SCEL' 'LEBL'
 'MMMX' 'LIRF' 'LFPG' 'EGLL' 'EDDF' 'LEMD' 'LIMC' 'FAOR' 'SUMU' 'SKBO'
 'SBUL' 'SBGO' 'SBMA' 'SBFI' 'SBBV' 'SBSG' 'SBJP' 'SBAR' 'SBPJ' 'OTHH'
 'SBRP' 'SWGN' 'SBQV' 'SWKO' 'SBTF' 'SWEI' 'SWPI' 'SBIH' 'KLAX' 'SGAS'
 'KIAD' 'KORD' 'KEWR' 'KIAH' 'OMDB' 'EHAM' 'PAN

## Data Reshaping

In [61]:
# Merge two DataFrames: flights, airports
# Purpose: append lat, long column on flights
aviation = pd.merge(flights, airports, left_on='origin', right_on='ICAO')
aviation

,company,flight,line_type,origin,destination,scheduled_arr,real_arr,scheduled_arr_Year,scheduled_arr_Month,scheduled_arr_Day,real_arr_Year,real_arr_Month,real_arr_Day,ICAO,name,iso_country,iso_region,lat,long
0,AZU,2497,N,SBMO,SBRF,2019-06-07 21:50:00,2019-06-07 21:39:00,2019,6,7,2019,6,7,SBMO,Zumbi dos Palmares Airport,BR,BR-AL,-9.510809898376465,-35.79169845581055
1,TAM,4794,N,SBMO,SBBR,2019-06-08 19:55:00,2019-06-08 19:45:00,2019,6,8,2019,6,8,SBMO,Zumbi dos Palmares Airport,BR,BR-AL,-9.510809898376465,-35.79169845581055
2,TAM,4765,N,SBMO,SBGR,2019-06-08 22:05:00,2019-06-08 21:47:00,2019,6,8,2019,6,8,SBMO,Zumbi dos Palmares Airport,BR,BR-AL,-9.510809898376465,-35.79169845581055
3,TAM,4575,N,SBMO,SBGR,2019-06-08 05:50:00,2019-06-08 05:54:00,2019,6,8,2019,6,8,SBMO,Zumbi dos Palmares Airport,BR,BR-AL,-9.510809898376465,-35.79169845581055
4,GLO,9592,N,SBMO,SBKP,2019-06-08 16:35:00,2019-06-08 16:59:00,2019,6,8,2019,6,8,SBMO,Zumbi dos Palmares Airport,BR,BR-AL,-9.510809898376465,-35.79169845581055
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1457578,AFR,0454,I,GOOY,SBGR,2019-03-04 08:43:00,2019-03-04 08:43:00,2019,3,4,2019,3,4,GOOY,LÃ©opold SÃ©dar Senghor International Airport,SN,SN-DK,14.739700317382812,-17.49020004272461
1457579,BOV,9730,I,SLOR,SBGL,2019-03-06 05:00:00,2019-03-06 05:00:00,2019,3,6,2019,3,6,SLOR,Juan Mendoza Airport,BO,BO-O,-17.962600708,-67.0762023926
1457580,AZU,9843,I,MSLP,KFLL,2019-04-14 17:20:00,2019-04-14 17:20:00,2019,4,14,2019,4,14,MSLP,MonseÃ±or Ãscar Arnulfo Romero International ...,SV,SV-PA,13.4409,-89.055702
1457581,UAE,0262,I,OEJN,OMDB,2019-04-11 20:17:00,2019-04-11 20:17:00,2019,4,11,2019,4,11,OEJN,King Abdulaziz International Airport,SA,SA-02,21.6796,39.156502


In [63]:
# Drop unnecessary columns
aviation.drop(['scheduled_arr', 'real_arr', 'scheduled_arr_Year', 'scheduled_arr_Month', 'scheduled_arr_Day', 'ICAO', 'iso_country', 'iso_region'], axis=1, inplace=True)
aviation

,company,flight,line_type,origin,destination,real_arr_Year,real_arr_Month,real_arr_Day,name,lat,long
0,AZU,2497,N,SBMO,SBRF,2019,6,7,Zumbi dos Palmares Airport,-9.510809898376465,-35.79169845581055
1,TAM,4794,N,SBMO,SBBR,2019,6,8,Zumbi dos Palmares Airport,-9.510809898376465,-35.79169845581055
2,TAM,4765,N,SBMO,SBGR,2019,6,8,Zumbi dos Palmares Airport,-9.510809898376465,-35.79169845581055
3,TAM,4575,N,SBMO,SBGR,2019,6,8,Zumbi dos Palmares Airport,-9.510809898376465,-35.79169845581055
4,GLO,9592,N,SBMO,SBKP,2019,6,8,Zumbi dos Palmares Airport,-9.510809898376465,-35.79169845581055
...,...,...,...,...,...,...,...,...,...,...,...
1457578,AFR,0454,I,GOOY,SBGR,2019,3,4,LÃ©opold SÃ©dar Senghor International Airport,14.739700317382812,-17.49020004272461
1457579,BOV,9730,I,SLOR,SBGL,2019,3,6,Juan Mendoza Airport,-17.962600708,-67.0762023926
1457580,AZU,9843,I,MSLP,KFLL,2019,4,14,MonseÃ±or Ãscar Arnulfo Romero International ...,13.4409,-89.055702
1457581,UAE,0262,I,OEJN,OMDB,2019,4,11,King Abdulaziz International Airport,21.6796,39.156502


In [20]:
flights_pt = pd.pivot_table(data=flights, values='flight', index='destination', columns='real_arr_Year', aggfunc='count')
flights_pt

real_arr_Year,2017,2018,2019,2020,2021
destination,,,,,
CYUL,NaN,NaN,9.0,38.0,NaN
CYYZ,NaN,NaN,363.0,90.0,NaN
EDDB,NaN,NaN,1.0,NaN,NaN
EDDF,NaN,NaN,1344.0,296.0,107.0
EDDM,NaN,NaN,9.0,33.0,NaN
...,...,...,...,...,...
YSSY,NaN,NaN,3.0,NaN,NaN
ZBAA,NaN,NaN,104.0,25.0,1.0
ZBHH,NaN,NaN,NaN,1.0,NaN
